In [1]:
import torch
from PIL import Image
import torchvision
import numpy as np
from detection_transforms import RandomHorizontalFlip, RandomVerticalFlip, RandomRotate, RandomResize, Compose, ToTensor

In [2]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
net = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = net.roi_heads.box_predictor.cls_score.in_features
net.roi_heads.box_predictor = FastRCNNPredictor(in_features, 7)

In [3]:
from data import BedsoreDataset
tfmc_train = Compose([
    ToTensor()
])

ds = BedsoreDataset('data', transforms=tfmc_train)
image,target = ds[3]

In [11]:
ds[3]

(tensor([[[0.6431, 0.7098, 0.6667,  ..., 0.3804, 0.3961, 0.4157],
          [0.6471, 0.7451, 0.7373,  ..., 0.3765, 0.3804, 0.3922],
          [0.6706, 0.7961, 0.8078,  ..., 0.3882, 0.3882, 0.4039],
          ...,
          [0.3922, 0.4235, 0.4431,  ..., 0.1098, 0.1137, 0.1176],
          [0.3922, 0.4275, 0.4510,  ..., 0.1059, 0.1098, 0.1176],
          [0.3961, 0.4314, 0.4549,  ..., 0.1059, 0.1098, 0.1216]],
 
         [[0.6157, 0.6824, 0.6392,  ..., 0.3059, 0.3098, 0.3294],
          [0.6235, 0.7216, 0.7098,  ..., 0.3020, 0.2941, 0.3059],
          [0.6471, 0.7725, 0.7843,  ..., 0.3098, 0.3098, 0.3176],
          ...,
          [0.2627, 0.2941, 0.3137,  ..., 0.0902, 0.0941, 0.0980],
          [0.2627, 0.2980, 0.3216,  ..., 0.0863, 0.0902, 0.0980],
          [0.2667, 0.3020, 0.3255,  ..., 0.0863, 0.0902, 0.1020]],
 
         [[0.5451, 0.6118, 0.5686,  ..., 0.2510, 0.2549, 0.2745],
          [0.5608, 0.6588, 0.6471,  ..., 0.2471, 0.2431, 0.2549],
          [0.5922, 0.7176, 0.7294,  ...,

In [5]:
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn

In [6]:
net = maskrcnn_resnet50_fpn()

In [7]:
net([image],[target])

/home/zrway/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/home/zrway/anaconda3/lib/python3.7/site-packages/torchvision/models/detection/_utils.py:321: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  match_quality_matrix == highest_quality_foreach_gt[:, None]


{'loss_classifier': tensor(4.4903, grad_fn=<NllLossBackward>),
 'loss_box_reg': tensor(0.0217, grad_fn=<DivBackward0>),
 'loss_mask': tensor(0.8139, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_objectness': tensor(0.6905, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(0.0025, grad_fn=<DivBackward0>)}